In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tqdm
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# Feature engineering practise 3

### **Step 1 :** Loading all the CSVs ###

In [3]:
train_csv = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
item_cats = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops_csv = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
items_csv = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
test_csv = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

### **Step 2.1** : Removing outliers from train csv ###

In [4]:
# Removing item_cnt_day > 1000
print("[*] Dataframe length before removing item_cnt rows{}".format(len(train_csv)))
train_csv.drop(train_csv[train_csv.item_cnt_day > 1000].index, inplace=True)
print(train_csv.head())
print("[*] Dataframe length after removing item_cnt rows{}".format(len(train_csv)))

# Removing item_cnt_day < 0 
print("[*] Dataframe length before removing item_cnt rows{}".format(len(train_csv)))
train_csv.drop(train_csv[train_csv.item_cnt_day < 0].index, inplace=True)
print(train_csv.head())
print("[*] Dataframe length after removing item_cnt rows{}".format(len(train_csv)))

# Removing items with price > 50K
print("[*] Dataframe length before removing price rows{}".format(len(train_csv)))
train_csv.drop(train_csv[train_csv.item_price > 50000].index, inplace=True)
print(train_csv.head())
print("[*] Dataframe length after removing price rows{}".format(len(train_csv)))



[*] Dataframe length before removing item_cnt rows2935849
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
[*] Dataframe length after removing item_cnt rows2935848
[*] Dataframe length before removing item_cnt rows2935848
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
5  10.01.2013

### **Step 2.2** : Feature engineering Shops csv ###

In [6]:
# getting the city names of thye shops 
shops_csv['city'] = shops_csv['shop_name'].apply(lambda x: x.split()[0])
# viewing the unique cities
print("[*] Unique Cities are :")
print(shops_csv["city"].unique())

# label encoding the city_column
# Create Label Encoder
label_encoder = LabelEncoder()
# City Feature Label Encoding 
shops_csv['city'] = label_encoder.fit_transform(shops_csv['city'])
shops_csv = shops_csv.drop('shop_name', axis=1)
print(shops_csv.head())

[*] Unique Cities are :
['!Якутск' 'Адыгея' 'Балашиха' 'Волжский' 'Вологда' 'Воронеж' 'Выездная'
 'Жуковский' 'Интернет-магазин' 'Казань' 'Калуга' 'Коломна' 'Красноярск'
 'Курск' 'Москва' 'Мытищи' 'Н.Новгород' 'Новосибирск' 'Омск'
 'РостовНаДону' 'СПб' 'Самара' 'Сергиев' 'Сургут' 'Томск' 'Тюмень' 'Уфа'
 'Химки' 'Цифровой' 'Чехов' 'Якутск' 'Ярославль']
   shop_id  city
0        0     0
1        1     0
2        2     1
3        3     2
4        4     3


### **Step 2.3** : Feature engineering of item-categories csv ###   

In [7]:
# Extract the first word of the item_categories_name into category
item_cats['category'] = item_cats['item_category_name'].apply(lambda x: x.split()[0]) 

# printing the categories and the number of unique categories present
print("[*] Item categories are \n", item_cats['category'].value_counts())

def make_etc(x):
    if len(item_cats[item_cats['category']==x]) >= 5:
        return x
    else:
        return 'etc'

# Replace with 'etc' if category count is less than 5
item_cats['category'] = item_cats['category'].apply(make_etc)

# Create Label Encoder
label_encoder = LabelEncoder()
# Category Feature Label Encoding 
item_cats['category'] = label_encoder.fit_transform(item_cats['category'])

# Remove item_category_name feature
item_cats = item_cats.drop('item_category_name', axis=1)
print(item_cats.head())

[*] Item categories are 
 Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Программы      6
Музыка         6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Доставка       1
Элементы       1
Билеты         1
Name: category, dtype: int64
   item_category_id  category
0                 0         0
1                 1         1
2                 2         1
3                 3         1
4                 4         1


### **Step 3 :** Merging train with test data ###

In [8]:
idx_features = ['date_block_num', 'shop_id', 'item_id'] # base features
# setting the test data as month 34 so that they are easy to filter at last
test_csv["date_block_num"] = 34
# Concatenate train and test
merged_csv = pd.concat([train_csv, test_csv.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)
# Replace NaN with 0
merged_csv = merged_csv.fillna(0)

merged_csv.head()



,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,06.01.2013,0,25,2554,1709.05,1.0
3,15.01.2013,0,25,2555,1099.00,1.0
4,10.01.2013,0,25,2564,349.00,1.0


### **Step 4** : Combining the other CSVs with the merged csv ###

In [9]:
# combining items_csv
merged_csv = pd.merge(merged_csv, items_csv, on = ['item_id'], how = "left")
merged_csv.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58
2,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58
3,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56
4,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59


In [10]:
# combining shops_csv with merged csv
merged_csv = pd.merge(merged_csv, shops_csv, on = ['shop_id'], how = "left")
merged_csv.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,city
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,31
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,14
2,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,14
3,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,14
4,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,14


In [11]:
# combining item_category csv_with merged_csv
merged_csv = pd.merge(merged_csv, item_cats, on = ['item_category_id'], how = "left")
merged_csv.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,city,category
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,31,5
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,14,7
2,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,14,7
3,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,14,7
4,10.01.2013,0,25,2564,349.00,1.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,14,7


### **Step 5** : Label encoding shop name, item name and item_category_name ###

In [12]:
# introducing label encoding to item name and shop name
from sklearn import preprocessing
merged_csv['item_name'] = preprocessing.LabelEncoder().fit_transform(merged_csv['item_name'])
merged_csv.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,city,category
0,02.01.2013,0,59,22154,999.00,1.0,22149,37,31,5
1,03.01.2013,0,25,2552,899.00,1.0,2552,58,14,7
2,06.01.2013,0,25,2554,1709.05,1.0,2554,58,14,7
3,15.01.2013,0,25,2555,1099.00,1.0,2555,56,14,7
4,10.01.2013,0,25,2564,349.00,1.0,2564,59,14,7


### **Step 6** : Creating a second csv where we have all the combination of shops and items for every month ###

In [14]:
# MOST IMPORTANT STEP OF ALL
# we need to predict item_cnt month so we need to create the feature item_cnt_month
# creating all possible combinations of month, shop and item
from itertools import product
train_data = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in merged_csv['date_block_num'].unique():
    sales = merged_csv[merged_csv.date_block_num == i]
    train_data.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype = np.int16))
    
train_data = pd.DataFrame(np.vstack(train_data), columns = cols )
train_data["date_block_num"] = train_data["date_block_num"].astype(np.int8)
train_data["shop_id"] = train_data["shop_id"].astype(np.int8)
train_data["item_id"] = train_data["item_id"].astype(np.int16)
train_data.sort_values( cols, inplace = True )
train_data.head()

,date_block_num,shop_id,item_id
139168,0,0,19
141406,0,0,27
144881,0,0,28
142572,0,0,29
138861,0,0,32


### **Step 7** : Introducing 2 new features namely monthly count of every item sold and their mean price to the new matrix created in Step 6 ###

In [15]:
# introduction of 2 new features. "item_cnt_month" and "item_price_mean"
idx_features = ["date_block_num", "shop_id", "item_id"]
group = merged_csv.groupby(idx_features).agg({'item_cnt_day': 'sum',
                                               'item_price': 'mean'})
group = group.reset_index()
group = group.rename(columns={'item_cnt_day': 'item_cnt_month', 'item_price': 'item_price_mean'})

train_data = train_data.merge(group, on=idx_features, how='left')
train_data["item_cnt_month"] = train_data["item_cnt_month"].fillna(0).astype(np.float16)
train_data["item_price_mean"] = train_data["item_price_mean"].fillna(0).astype(np.float16)
train_data.head(10)

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean
0,0,0,19,0.0,0.0
1,0,0,27,0.0,0.0
2,0,0,28,0.0,0.0
3,0,0,29,0.0,0.0
4,0,0,32,6.0,221.0
5,0,0,33,3.0,347.0
6,0,0,34,0.0,0.0
7,0,0,35,1.0,247.0
8,0,0,40,0.0,0.0
9,0,0,41,0.0,0.0


### **Step 8** : Merging shops and items ### 

In [16]:
train_data = pd.merge(train_data, shops_csv, on = ['shop_id'], how = "left")
train_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city
0,0,0,19,0.0,0.0,0
1,0,0,27,0.0,0.0,0
2,0,0,28,0.0,0.0,0
3,0,0,29,0.0,0.0,0
4,0,0,32,6.0,221.0,0


In [17]:
train_data = pd.merge(train_data, items_csv, on = ['item_id'], how = "left")
train_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_name,item_category_id
0,0,0,19,0.0,0.0,0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
1,0,0,27,0.0,0.0,0,"007 Legends [PS3, русская версия]",19
2,0,0,28,0.0,0.0,0,"007 Legends [PС, Jewel, русская версия]",30
3,0,0,29,0.0,0.0,0,"007 Legends [Xbox 360, русская версия]",23
4,0,0,32,6.0,221.0,0,1+1,40


In [18]:
train_data = pd.merge(train_data, item_cats, on = ['item_category_id'], how = "left")
train_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_name,item_category_id,category
0,0,0,19,0.0,0.0,0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,5
1,0,0,27,0.0,0.0,0,"007 Legends [PS3, русская версия]",19,3
2,0,0,28,0.0,0.0,0,"007 Legends [PС, Jewel, русская версия]",30,3
3,0,0,29,0.0,0.0,0,"007 Legends [Xbox 360, русская версия]",23,3
4,0,0,32,6.0,221.0,0,1+1,40,5


In [19]:
# dropping item_name from train data
train_data = train_data.drop("item_name", axis = 1)
train_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_category_id,category
0,0,0,19,0.0,0.0,0,40,5
1,0,0,27,0.0,0.0,0,19,3
2,0,0,28,0.0,0.0,0,30,3
3,0,0,29,0.0,0.0,0,23,3
4,0,0,32,6.0,221.0,0,40,5


# Introduction of lag features

In [20]:
# 1st I want to create a lag feature column for my item_cnt_mnth
def creating_lag_features(df, lags, cols):
    for col in cols:
        temp = df[["date_block_num", "shop_id","item_id", col]]
        shifted = temp.copy()
        for lag in lags:
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+ str(lag)]
            shifted.date_block_num = shifted.date_block_num + lag
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [22]:
# creating lag features w.r.t "item_cnt_month"
new_matrix_lagged = creating_lag_features(train_data, [1, 2, 3], ["item_cnt_month"])
new_matrix_lagged.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_category_id,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,0,19,0.0,0.0,0,40,5,NaN,NaN,NaN
1,0,0,27,0.0,0.0,0,19,3,NaN,NaN,NaN
2,0,0,28,0.0,0.0,0,30,3,NaN,NaN,NaN
3,0,0,29,0.0,0.0,0,23,3,NaN,NaN,NaN
4,0,0,32,6.0,221.0,0,40,5,NaN,NaN,NaN


In [23]:
# creating lag features w.r.t item_price
new_matrix_lagged = creating_lag_features(new_matrix_lagged, [1, 2, 3], ["item_price_mean"])
new_matrix_lagged.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_category_id,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_price_mean_lag_1,item_price_mean_lag_2,item_price_mean_lag_3
0,0,0,19,0.0,0.0,0,40,5,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0.0,0.0,0,19,3,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0.0,0.0,0,30,3,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0.0,0.0,0,23,3,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,32,6.0,221.0,0,40,5,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# replacing all NaN with 0
new_matrix_lagged = new_matrix_lagged.fillna(0).astype(np.float16)
new_matrix_lagged.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_mean,city,item_category_id,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_price_mean_lag_1,item_price_mean_lag_2,item_price_mean_lag_3
0,0.0,0.0,19.0,0.0,0.0,0.0,40.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,27.0,0.0,0.0,0.0,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,28.0,0.0,0.0,0.0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,29.0,0.0,0.0,0.0,23.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,32.0,6.0,221.0,0.0,40.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creating some visualization to observe the new csv that has been created

In [25]:
item_id = 32
visual_group1 = new_matrix_lagged.loc[new_matrix_lagged["item_id"] == item_id]
visual_group1a = visual_group1.groupby(['date_block_num' , 'item_id']).agg({"item_cnt_month" : "sum", "item_price_mean" : "sum"}).reset_index()
visual_group1a = visual_group1a.rename(columns={'item_cnt_month': 'added_item_cnt_month', 'item_price_mean': 'added_item_price_mean'})
print(visual_group1a.head())

# plotting the graph
fig = px.line(visual_group1a, x="date_block_num", y=["added_item_cnt_month"], title='Sale record for item_id {}'.format(item_id))
fig.show()

   date_block_num  item_id  added_item_cnt_month  added_item_price_mean
0             0.0     32.0                 299.0                12400.0
1             1.0     32.0                 208.0                12400.0
2             2.0     32.0                 178.0                13568.0
3             3.0     32.0                  97.0                11960.0
4             4.0     32.0                  66.0                10688.0


# Creating dataset and defining model and training

In [ ]:
# Not used at the moment
"""
from sklearn.model_selection import train_test_split
# here we are going to use new matrix lagged as the training set
# creating the dataset 
train, test = train_test_split(new_matrix_lagged, test_size=0.2)

y_train = train["item_cnt_month"]
x_train = train.drop(["item_cnt_month"], axis = 1)

y_test = test["item_cnt_month"]
x_test = test.drop(["item_cnt_month"], axis = 1)

print("X_train : \n", x_train.head())
print("\n")
print("y_train : \n", y_train.head())
print("[*] Length of X_train : {}".format(x_train.shape))
print("[*] Length of X_test : {}".format(x_test.shape))
"""



In [26]:
"""
As we have train and test merged and we have given the merged test data the date_block_num = 34,
hence we use this method to separate our train and test data
"""
X_train = new_matrix_lagged[new_matrix_lagged['date_block_num'] < 33]
X_train = X_train.drop(['item_cnt_month'], axis=1)
# Valid data (Features)
X_valid = new_matrix_lagged[new_matrix_lagged['date_block_num'] == 33]
X_valid = X_valid.drop(['item_cnt_month'], axis=1)
# Test data (Features)
X_test = new_matrix_lagged[new_matrix_lagged['date_block_num'] == 34]
X_test = X_test.drop(['item_cnt_month'], axis=1)

# Train data (Target values)
y_train = new_matrix_lagged[new_matrix_lagged['date_block_num'] < 33]['item_cnt_month']
# Valid data (Target values)
y_valid = new_matrix_lagged[new_matrix_lagged['date_block_num'] == 33]['item_cnt_month']


In [27]:
print("[*] x-train shape : {}".format(X_train.shape))
print("[*] y-train shape : {}".format(y_train.shape))
print("[*] x-test shape : {}".format(X_test.shape))
print("[*] x-valid shape : {}".format(X_valid.shape))
print("[*] y-valid shape : {}".format(y_valid.shape))



[*] x-train shape : (10646376, 13)
[*] y-train shape : (10646376,)
[*] x-test shape : (214200, 13)
[*] x-valid shape : (238084, 13)
[*] y-valid shape : (238084,)


# Implementing LightGBM

In [28]:
import lightgbm as lgb

# lgb hyper-parameters
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}
# currentlz no categorical features
#cat_features = ['shop_id', 'city', 'item_category_id', 'category', 'month']

# lgb train and valid dataset
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)
 
# Train LightGBM model
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=2000,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      
                      verbose_eval=100)  

lgb_model.save_model('/kaggle/working/best_model.txt', num_iteration=lgb_model.best_iteration)

[LightGBM] [Info] Total Bins 1660
[LightGBM] [Info] Number of data points in the train set: 10646376, number of used features: 13
[LightGBM] [Info] Start training from score 0.336693
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 2.63971	valid_1's rmse: 2.16911
[200]	training's rmse: 2.25706	valid_1's rmse: 1.96483
[300]	training's rmse: 2.05898	valid_1's rmse: 1.92265
[400]	training's rmse: 1.94487	valid_1's rmse: 1.92832
[500]	training's rmse: 1.86896	valid_1's rmse: 1.9691
Early stopping, best iteration is:
[359]	training's rmse: 1.98398	valid_1's rmse: 1.9139


In [29]:
submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
preds = lgb_model.predict(X_test).clip(0,20)

submission['item_cnt_month'] = preds
print(submission.head())
submission.to_csv('/kaggle/working/submission.csv', index=False)

   ID  item_cnt_month
0   0        0.057808
1   1        0.086972
2   2        0.074569
3   3        0.057808
4   4        0.057808
